In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import dump
import pathlib
from joblib import load


2023-12-17 21:35:18.295822: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
starting_time_step = datetime(year=2016, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 1900, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
dformer = DataTransformer(env_config=env_config)
dfs = dformer.create_input_dataframes()
market = MarketSimulator(env_config=env_config, data_frames=dfs)
market.reset()
print("here", market.the_current_time_step, market.shares_data)

In [ ]:
nw_starting_time_step = datetime(year=2023, month=9, day=1)

nw_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nw_env_config = {
    "db_params": nw_db_params, 
    "max_episode_steps": 60, 
    "the_current_time_step": nw_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 60,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nw_dformer = DataTransformer(env_config=nw_env_config)
nw_dfs = nw_dformer.create_input_dataframes()
# display(nw_dfs[2])
nw_market = MarketSimulator(env_config=nw_env_config, data_frames=nw_dfs)
nw_market.reset()
nw_forest = Forester(market=nw_market)
print(nw_market.the_current_time_step, nw_market.shares_data)


In [ ]:
pd.set_option('display.max_rows', None)
display(nw_dfs[2])

In [ ]:
forest = Forester(market=market)
forest.make_forest_data()
X_train = forest.train_input
y_train = forest.train_output
y_test = forest.test_output
X_test = forest.test_input
print(forest.market.the_current_time_step)

nw_forest.make_forest_data()
nw_X_train = nw_forest.train_input
nw_y_train = nw_forest.train_output
nw_y_test = nw_forest.test_output
nw_X_test = nw_forest.test_input
print(nw_forest.market.the_current_time_step)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [73]
# max_depth = [int(x) for x in np.linspace(10, 80, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5]
# min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

best_params = {
    'n_estimators': None,
    'max_features': None,
    'max_depth': None,
    'min_samples_split': None,
    'min_samples_leaf': None,
    'bootstrap': None,
    "accuracy": None
}

best_accuracy = 0

In [ ]:
iters = 1
for n_ests in n_estimators:
    for max_feat in max_features:
        for max_dept in max_depth:
            for min_sample_sp in min_samples_split:
                for min_sample_lf in min_samples_leaf:
                    for bt_strp in bootstrap:
                        rf = RandomForestClassifier(
                            n_estimators=n_ests,
                            max_features=max_feat,
                            max_depth=max_dept,
                            min_samples_split=min_sample_sp,
                            min_samples_leaf=min_sample_lf,
                            bootstrap=bt_strp
                        )
                        rf.fit(X_train, y_train)
                        nw_X_test = nw_forest.train_input
                        nw_y_pred = rf.predict(nw_X_test)

                        nw_y_test = nw_forest.train_output
                        nw_y_test_first_column = [row[1] for row in nw_y_test]
                        nw_y_pred_first_column = [row[1] for row in nw_y_pred]
                        accuracy = accuracy_score(nw_y_test_first_column, nw_y_pred_first_column)
                        print(f"{iters} Accuracy:", accuracy)
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params["n_estimators"]=n_ests
                            best_params["max_features"]=max_feat
                            best_params["max_depth"]=max_dept
                            best_params["min_samples_split"]=min_sample_sp
                            best_params["min_samples_leaf"]=min_sample_lf
                            best_params["bootstrap"]=bt_strp
                            best_params["accuracy"] = accuracy
                        iters = iters + 1

In [ ]:
print(best_params)

In [ ]:
# rf = RandomForestClassifier(
#     n_estimators=best_params["n_estimators"],
#     max_features=best_params["max_features"],
#     max_depth=best_params["max_depth"],
#     min_samples_split=best_params["min_samples_split"],
#     min_samples_leaf=best_params["min_samples_leaf"],
#     bootstrap=best_params["bootstrap"]
# )

rf = RandomForestClassifier(
    n_estimators=533,
    max_features="sqrt",
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=False
)

(current_file_path, new_file_path) = DataTransformer.get_new_file_path()
rf.fit(X_train, y_train)  # Your existing code

dump(rf, new_file_path)

In [2]:
(current_file_path, new_file_path) = DataTransformer.get_new_file_path()

rf = load(pathlib.Path(current_file_path))
nwa_starting_time_step = datetime(year=2023, month=6, day=1)

nwa_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwa_env_config = {
    "db_params": nwa_db_params, 
    "max_episode_steps": 100, 
    "the_current_time_step": nwa_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nwa_dformer = DataTransformer(env_config=nwa_env_config)
nwa_dfs = nwa_dformer.create_input_dataframes()
# display(nwa_dfs[2])
nwa_market = MarketSimulator(env_config=nwa_env_config, data_frames=nwa_dfs)
nwa_market.reset()
nwa_forest = Forester(market=nwa_market)

nwa_forest.make_forest_data()

nwa_X_train = nwa_forest.train_input
nwa_y_train = nwa_forest.train_output

nwa_y_pred = rf.predict(nwa_X_train)

nwa_y_test = nwa_forest.train_output
predictable_stocks = {}
for inx in range(100):
    nwa_y_test_first_column = [row[inx] for row in nwa_y_test]
    nwa_y_pred_first_column = [row[inx] for row in nwa_y_pred]

    accuracy = accuracy_score(nwa_y_test_first_column, nwa_y_pred_first_column)
    st_indx = inx+1
    print(st_indx, "Accuracy:", accuracy)
    predictable_stocks[st_indx] = accuracy

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:283: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:284: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:313: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

1 Accuracy: 0.22580645161290322
2 Accuracy: 0.20430107526881722
3 Accuracy: 0.3010752688172043
4 Accuracy: 0.8064516129032258
5 Accuracy: 0.22580645161290322
6 Accuracy: 0.25806451612903225
7 Accuracy: 0.3763440860215054
8 Accuracy: 0.24731182795698925
9 Accuracy: 0.21505376344086022
10 Accuracy: 0.1935483870967742
11 Accuracy: 0.3010752688172043
12 Accuracy: 0.24731182795698925
13 Accuracy: 0.3333333333333333
14 Accuracy: 0.45161290322580644
15 Accuracy: 0.20430107526881722
16 Accuracy: 0.27956989247311825
17 Accuracy: 0.6881720430107527
18 Accuracy: 0.6236559139784946
19 Accuracy: 0.22580645161290322
20 Accuracy: 0.5053763440860215
21 Accuracy: 0.25806451612903225
22 Accuracy: 0.20430107526881722
23 Accuracy: 0.22580645161290322
24 Accuracy: 0.5806451612903226
25 Accuracy: 0.20430107526881722
26 Accuracy: 0.46236559139784944
27 Accuracy: 0.1827956989247312
28 Accuracy: 0.23655913978494625
29 Accuracy: 0.22580645161290322
30 Accuracy: 0.27956989247311825
31 Accuracy: 0.204301075268817

In [3]:
sorted_stocks = dict(sorted(predictable_stocks.items(), key=lambda item: item[1], reverse=True))

# Print the sorted dictionary
df = pd.DataFrame(list(sorted_stocks.items()), columns=['Key', 'Value'])
pd.set_option('display.max_rows', None)
# Print the DataFrame
top_17 = df.head(17)['Key'].tolist()
print(top_17)
display(df)

[4, 79, 59, 17, 53, 54, 18, 24, 68, 95, 35, 80, 87, 20, 26, 14, 78]


,Key,Value
0,4,0.806452
1,79,0.763441
2,59,0.752688
3,17,0.688172
4,53,0.655914
5,54,0.655914
6,18,0.623656
7,24,0.580645
8,68,0.580645
9,95,0.569892


In [ ]:
(current_file_path, new_file_path) = DataTransformer.get_new_file_path()

rf = load(pathlib.Path(current_file_path))

nwb_starting_time_step = datetime(year=2023, month=1, day=1)

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True,
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
# display(nwb_dfs[2])
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_forest = Forester(market=nwb_market)

nwb_forest.make_forest_data()

total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step)
for indx in range(100):
    if indx < 7:
        acts = np.full(101, 10)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        flat_state = np.array([nwb_forest.get_flattened_states(nwb_market.state)])
        my_pred = rf.predict(flat_state)
        acts = my_pred[0]
        # acts = np.full(101, 10)
        rand_acts = nwb_forest.act_random(acts, use_best_stks=False)
        nwb_market.step(rand_acts)

print(nwb_market.wallet_state)

In [ ]:
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
gain = final_total_potfolio - initial_total_potfolio
print("gain", gain)